## SESIÓN 3: RESOLUCIÓN DE PROBLEMAS CON BÚSQUEDA HEURÍSTICA

**Carmen Miguel Spínola**

**Miguel Ángel Molina de la Rosa**

### IMPLEMENTACIÓN

#### Versión 1 : Representación de MoVIX con el tablero entero como estado

In [28]:
from random import sample
from search import *

class MovIX_Puzzle1(Problem):
    """MovIX. Cada estado son N tuplas que cada una se corresponde con una fila donde se guarda lo que hay en cada casilla,
    representando las celdas vacías del tablero (representado por ' ') o conteniendo una ficha ('S', 'L', 'V', 'H'). 
    Las posiciones corresponden con las posiciones del tablero."""

    def __init__(self, N, X, Y, initial=None):
        """
        N: Tamaño del tablero (NxN).
        X: Número de fichas a alinear.
        Y: Número de fichas de cada tipo a colocar (S, L, V, H).
        initial: Estado inicial opcional (si no se da, se genera aleatoriamente).
        """
        self.N = N
        self.X = X
        self.Y = Y
        if initial:
            self.initial = initial
        else:
            self.initial = self.generar_tablero(N, Y)

    #Función que genera el tablero de forma aleatoria
    def generar_tablero(self, N, Y):
        """
        N: Tamaño del tablero (NxN).
        Y: Número de fichas de cada tipo a colocar (S, L, V, H).
        """
        # Definir las piezas: Y fichas de cada tipo
        piezas = ['S', 'L', 'V', 'H'] * Y
        num_piezas = len(piezas)
        # Crear un tablero vacío con N*N celdas
        tablero = [' '] * (N * N)
        
        # Elegir posiciones aleatorias para las piezas
        posiciones = sample(range(N * N), num_piezas)
        
        # Colocar las piezas en las posiciones seleccionadas
        for i, pos in enumerate(posiciones):
            tablero[pos] = piezas[i]
        
        # Convertir la lista en una lista de tuplas que representa el tablero
        tablero_final = [tuple(tablero[i:i + N]) for i in range(0, len(tablero), N)]
        
        return tuple(tablero_final)

    def actions(self, casilla):
        """Retorna una lista de las posibles acciones que puede hacer una ficha"""
        actions = []
        N = self.N
        for fila in range(N):
            for columna in range(N): 
                ficha = casilla[fila][columna]
                if ficha in ['S', 'L', 'V', 'H']:
                    #nos fijamos en los posibles movimientos si hay ficha
                    posibles_movs = self.obtener_mov(ficha, fila, columna, casilla)
                    for movimiento in posibles_movs:
                        a_fila, a_columna = movimiento
                        actions.append(('movimiento', fila, columna, a_fila, a_columna)) #fila, columna (casilla de partida) a_fila,a_columna(casilla a ocupar)
        return actions

    #Dada una ficha y su posición a que casillas puede moverse
    def obtener_mov(self, ficha, fila, columna, casilla):
        N = self.N
        movs = []
        
        if ficha == 'L':  # Ficha lenta
            #Se puede mover abajo, izquierda derecha, arriba y en las diagonales
            direcciones = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
            #comprobamos cada mov disponible
            for dir_fila, dir_col in direcciones:
                nueva_fila, nueva_col = fila + dir_fila, columna + dir_col
                if 0 <= nueva_fila < N and 0 <= nueva_col < N: #comprueba la pertenencia al tablero
                    if casilla[nueva_fila][nueva_col] == ' ':
                        movs.append((nueva_fila, nueva_col))

        elif ficha == 'V':  # Ficha vertical
            for dir_fila in range(-N + 1, N):
                if dir_fila == 0: # el 0 no lo miramos porque es la posicion en la que estamos
                    continue
                nueva_fila = fila + dir_fila
                if 0 <= nueva_fila < N:
                    paso = 1 if nueva_fila > fila else -1 #dependiendo del signo iremos arriba o abajo
                    camino_despejado = True
                    
                    for r in range(fila + paso, nueva_fila, paso):
                        if casilla[r][columna] != ' ':
                            camino_despejado = False
                            break  # Si encontramos una ficha en el camino, no está despejado
                    
                    if camino_despejado:
                        movs.append((nueva_fila, columna))

        elif ficha == 'H':  # Ficha horizontal
            for dir_col in range(-N + 1, N):
                if dir_col == 0:
                    continue
                nueva_col = columna + dir_col
                if 0 <= nueva_col < N:
                    paso = 1 if nueva_col > columna else -1
                    camino_despejado = True
                    
                    for c in range(columna + paso, nueva_col, paso):
                        if casilla[fila][c] != ' ':
                            camino_despejado = False
                            break  # Si encontramos una ficha en el camino, no está despejado
                    
                    if camino_despejado:
                        movs.append((fila, nueva_col))

        elif ficha == 'S':  # Ficha saltadora
            # La ficha Saltadora ('S') puede saltar sobre una ficha adyacente
            # y aterrizar en la siguiente celda vacía (horizontal o vertical).
            direcciones = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Arriba, Abajo, Izquierda, Derecha
            for dir_fila, dir_col in direcciones:
                fila_adyacente, col_adyacente = fila + dir_fila, columna + dir_col
                fila_salto, col_salto = fila + 2 * dir_fila, columna + 2 * dir_col
                
                if (0 <= fila_adyacente < N) and (0 <= col_adyacente < N) and (0 <= fila_salto < N) and (0 <= col_salto < N):
                    if casilla[fila_adyacente][col_adyacente] in ['S', 'L', 'V', 'H']:
                        if casilla[fila_salto][col_salto] == ' ':
                            movs.append((fila_salto, col_salto))

        return movs
        
    def result(self, estado, accion):
        """Aplica una acción en un estado y devuelve el nuevo estado."""
        l = [list(fila) for fila in estado]  # Convertir el estado a una lista 
        ficha, fila, columna, nueva_fila, nueva_columna = accion

        # Mover la ficha desde la posición actual a la nueva posición
        l[nueva_fila][nueva_columna] = l[fila][columna]
        l[fila][columna] = ' '  # Dejar vacío el lugar original

        # Convertir de vuelta a tupla y devolver el nuevo estado
        nuevo_estado = tuple(tuple(fila) for fila in l)
        return nuevo_estado

    def goal_test(self, casilla):
        """Verifica si hay una alineación de X fichas."""
        N = self.N  # Tamaño del tablero
        X = self.X  # Número de fichas a alinear
        piezas = ['S', 'L', 'V', 'H']  # Tipos de fichas

        # Comprobar filas
        for fila in range(N):
            for col in range(N - X + 1):  # Ajustar el rango para evitar salir de los límites
                # Extraer una línea horizontal de X fichas
                if col + X <= N:  # Verificar que no excedemos el límite de la fila
                    linea = casilla[fila][col:col + X]
                    if all(celda in piezas for celda in linea):
                        return True  # Alineación encontrada en una fila

        # Comprobar columnas
        for col in range(N):
            for fila in range(N - X + 1):
                # Extraer una línea vertical de X fichas
                linea = [casilla[fila + i][col] for i in range(X)]
                if all(celda in piezas for celda in linea):
                    return True  # Alineación encontrada en una columna

        # Comprobar diagonales de izquierda a derecha (arriba-izquierda a abajo-derecha)
        for fila in range(N - X + 1):
            for col in range(N - X + 1):
                # Extraer una diagonal de X fichas
                linea = [casilla[fila + i][col + i] for i in range(X)]
                if all(celda in piezas for celda in linea):
                    return True  # Alineación encontrada en una diagonal 

        # Comprobar diagonales de derecha a izquierda (arriba-derecha a abajo-izquierda)
        for fila in range(N - X + 1):
            for col in range(X - 1, N):
                # Extraer una diagonal inversa de X fichas
                linea = [casilla[fila + i][col - i] for i in range(X)]
                if all(celda in piezas for celda in linea):
                    return True  # Alineación encontrada en una diagonal inversa 



#### Búsquedas ciegas con la primera versión

In [29]:

estado_inicial = (
 (' ', ' ', ' ', ' ', ' '),
 (' ', 'S', ' ', 'H', ' '),
 (' ', ' ', 'L', ' ', ' '),
 ('V', ' ', ' ', ' ', ' '),
 (' ', ' ', ' ',' ',' ')
)
#estado_inicial = None  # Se generará un tablero aleatorio si es None
puzzle = MovIX_Puzzle1(N=5, X=4, Y=1, initial=estado_inicial)

#resultado = breadth_first_tree_search(puzzle) #búsqueda en anchura sin control de repetidos
#resultado = breadth_first_graph_search(puzzle) #búsqueda en anchura con control de repetidos
#resultado = depth_first_tree_search(puzzle) #búsqueda en profundidad sin control de repetidos
resultado = depth_first_graph_search (puzzle) #búsqueda en profundidad con control de repetidos

if resultado:
    print("Se ha encontrado una solución")
    print("Camino a la solución:", resultado.solution())
else:
    print("No se ha encontrado una solución.")

Se ha encontrado una solución
Camino a la solución: [('movimiento', 3, 0, 4, 0), ('movimiento', 2, 2, 3, 3), ('movimiento', 4, 0, 2, 0), ('movimiento', 3, 3, 4, 4), ('movimiento', 2, 0, 3, 0), ('movimiento', 4, 4, 4, 3), ('movimiento', 3, 0, 1, 0), ('movimiento', 4, 3, 3, 4), ('movimiento', 3, 4, 2, 4), ('movimiento', 2, 4, 1, 4), ('movimiento', 1, 4, 0, 4), ('movimiento', 1, 3, 1, 4), ('movimiento', 1, 0, 4, 0), ('movimiento', 1, 4, 1, 2), ('movimiento', 4, 0, 3, 0), ('movimiento', 1, 1, 1, 3), ('movimiento', 3, 0, 2, 0), ('movimiento', 1, 2, 1, 1), ('movimiento', 2, 0, 4, 0), ('movimiento', 1, 1, 1, 0), ('movimiento', 0, 4, 1, 4), ('movimiento', 4, 0, 3, 0), ('movimiento', 1, 4, 2, 4), ('movimiento', 3, 0, 2, 0), ('movimiento', 2, 4, 3, 4), ('movimiento', 3, 4, 4, 4), ('movimiento', 2, 0, 4, 0), ('movimiento', 4, 4, 4, 3), ('movimiento', 4, 3, 3, 2), ('movimiento', 4, 0, 3, 0), ('movimiento', 3, 2, 4, 1), ('movimiento', 4, 1, 4, 0), ('movimiento', 3, 0, 2, 0), ('movimiento', 4, 0, 3,

##### Análisis de Búsquedas Ciegas

**Factor de ramificación: r<=(N-1+N-1+8+8)**

##### Búsqueda en profundidad:

- **Sin control de repetidos**: 
    - En el caso del estado inicial fácil, no consigue resolverlo por el orden en el que definimos las acciones (mirando primero a donde debe moverse la saltadora, que en la solución más óptima no se mueve).
    - Cuando el tablero inicial se genera al azar, vemos que el algoritmo es incompleto ya que pueden generarse bucles en el árbol de búxqueda.

- **Con control de repetidos**: 
    - Resuelve el problema con el estado inicial del enunciado pero nos da una solución subóptima.
    - En el resto de casos, el algoritmo sigue resultando ineficiente (a pesar de disminuir el tiempo de resolución) e incompleto.

##### Búsqueda en anchura:

- **Sin control de repetidos**: 
    - Resuelve el problema con el estado inicial dado de manera óptima en cuanto al número de pasos desde la raíz (2).
    - En el resto de casos, el algoritmo es completo aunque ineficiente. En algún momento nos dará una solución, aunque el estado final más cercano puede estar en un nivel muy profundo del árbol.

- **Con control de repetidos**: 
    - Resuelve de la misma forma el problema dado el estado inicial.
    - Es completo y mejora la eficiencia respecto al anterior algoritmo, pero en muchos casos tardará aún demasiado en terminar.

#### Versión 2 : Representación de MoVIX con las posiciones ocupadas (con la ficha que ocupa cada una) como estado

In [30]:
class MovIX_Puzzle2(Problem):
    """MovIX. Los estados son listas de tuplas que solo contienen posiciones ocupadas por fichas y la propia ficha que la ocupa."""

    def __init__(self, N, X, Y, initial=None):
        """
        :param N: Tamaño del tablero (NxN).
        :param X: Número de fichas a alinear.
        :param Y: Número de fichas de cada tipo a colocar (S, L, V, H).
        :param initial: Estado inicial opcional (si no se da, se genera aleatoriamente).
        """
        self.N = N
        self.X = X
        self.Y = Y
        if initial:
            self.initial = tuple(initial)
        else:
            self.initial = self.generar_estado_inicial(N, Y)

    def generar_estado_inicial(self, N, Y):
        """Genera un estado inicial con las posiciones ocupadas por las fichas."""
        piezas = ['S', 'L', 'V', 'H'] * Y
        posiciones = sample(range(N * N), len(piezas))
        estado = [(piezas[i], posiciones[i] // N, posiciones[i] % N) for i in range(len(piezas))]
        return tuple(estado)

    def actions(self, estado):
        """Retorna una lista de las posibles acciones que puede hacer una ficha."""
        actions = []
        for (ficha, fila, columna) in estado:
            posibles_movs = self.obtener_mov(ficha, fila, columna, estado)
            for movimiento in posibles_movs:
                a_fila, a_columna = movimiento
                actions.append(('movimiento', fila, columna, a_fila, a_columna))
        return actions

    def obtener_mov(self, ficha, fila, columna, estado):
        """Obtiene los movimientos válidos para una ficha en una posición específica."""
        N = self.N
        posiciones_ocupadas = [(f[1], f[2]) for f in estado]
        movs = []
        
        if ficha == 'L':  # Ficha lenta (se puede mover en todas las direcciones adyacentes)
            direcciones = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
            for dir_fila, dir_col in direcciones:
                nueva_fila, nueva_col = fila + dir_fila, columna + dir_col
                if 0 <= nueva_fila < N and 0 <= nueva_col < N and (nueva_fila, nueva_col) not in posiciones_ocupadas:
                    movs.append((nueva_fila, nueva_col))

        elif ficha == 'V':  # Ficha vertical
            for dir_fila in range(-N + 1, N):
                if dir_fila == 0: continue
                nueva_fila = fila + dir_fila
                if 0 <= nueva_fila < N:
                    paso = 1 if nueva_fila > fila else -1
                    camino_despejado = all((r, columna) not in posiciones_ocupadas for r in range(fila + paso, nueva_fila, paso))
                    if camino_despejado:
                        movs.append((nueva_fila, columna))

        elif ficha == 'H':  # Ficha horizontal
            for dir_col in range(-N + 1, N):
                if dir_col == 0: continue
                nueva_col = columna + dir_col
                if 0 <= nueva_col < N:
                    paso = 1 if nueva_col > columna else -1
                    camino_despejado = all((fila, c) not in posiciones_ocupadas for c in range(columna + paso, nueva_col, paso))
                    if camino_despejado:
                        movs.append((fila, nueva_col))

        elif ficha == 'S':  # Ficha saltadora
            direcciones = [(-1, 0), (1, 0), (0, -1), (0, 1)]
            for dir_fila, dir_col in direcciones:
                fila_adyacente, col_adyacente = fila + dir_fila, columna + dir_col
                fila_salto, col_salto = fila + 2 * dir_fila, columna + 2 * dir_col
                if (0 <= fila_adyacente < N and 0 <= col_adyacente < N and
                    0 <= fila_salto < N and 0 <= col_salto < N and 
                    (fila_adyacente, col_adyacente) in posiciones_ocupadas and 
                    (fila_salto, col_salto) not in posiciones_ocupadas):
                    movs.append((fila_salto, col_salto))

        return movs
        
    def result(self, estado, accion):
        """Aplica una acción en un estado y devuelve el nuevo estado."""
        ficha, fila, columna, nueva_fila, nueva_columna = accion
        nuevo_estado = [(f, r, c) for (f, r, c) in estado if not (r == fila and c == columna)]
        nuevo_estado.append((ficha, nueva_fila, nueva_columna))
        return tuple(nuevo_estado)

    def goal_test(self, estado):
        """Verifica si hay una alineación de X fichas de cualquier tipo."""
        N = self.N
        X = self.X

        # Recoger todas las posiciones ocupadas por fichas ('S', 'L', 'V', 'H')
        posiciones_ocupadas = [(fila, columna) for (ficha, fila, columna) in estado]

        # Recorrer todas las posiciones en la lista
        for fila, columna in posiciones_ocupadas:

            # Comprobar alineación horizontal (X fichas consecutivas en la misma fila)
            alineado_horizontal = True
            for i in range(X):
                if (fila, columna + i) not in posiciones_ocupadas or columna + i >= N:
                    alineado_horizontal = False
                    break
            if alineado_horizontal:
                return True

            # Comprobar alineación vertical
            alineado_vertical = True
            for i in range(X):
                if (fila + i, columna) not in posiciones_ocupadas or fila + i >= N:
                    alineado_vertical = False
                    break
            if alineado_vertical:
                return True

            # Comprobar alineación diagonal de izquierda a derecha 
            alineado_diagonal_izq_der = True
            for i in range(X):
                if (fila + i, columna + i) not in posiciones_ocupadas or fila + i >= N or columna + i >= N:
                    alineado_diagonal_izq_der = False
                    break
            if alineado_diagonal_izq_der:
                return True

            # Comprobar alineación diagonal de derecha a izquierda 
            alineado_diagonal_der_izq = True
            for i in range(X):
                if (fila + i, columna - i) not in posiciones_ocupadas or fila + i >= N or columna - i < 0:
                    alineado_diagonal_der_izq = False
                    break
            if alineado_diagonal_der_izq:
                return True

        # Si no se encuentra ninguna alineación, retornar False
        return False


#### Búsquedas ciegas con la segunda versión

In [31]:
"""estado_inicial = [
    ('S', 1, 1),  # Ficha 'S' en la fila 0, columna 0
    ('L', 2, 2),  # Ficha 'L' en la fila 1, columna 1
    ('V', 3, 0),  # Ficha 'V' en la fila 2, columna 2
    ('H', 1, 3)   # Ficha 'H' en la fila 3, columna 3
]"""
estado_inicial=None
puzzle = MovIX_Puzzle2(N=5, X=4, Y=1, initial=estado_inicial)


resultado = breadth_first_tree_search(puzzle) #búsqueda en anchura sin control de repetidos
#resultado = breadth_first_graph_search(puzzle) #búsqueda en anchura con control de repetidos
#resultado = depth_first_tree_search(puzzle) #búsqueda en profundidad sin control de repetidos
#resultado = depth_first_graph_search (puzzle) #búsqueda en profundidad con control de repetidos

if resultado:
    print("Se ha encontrado una solución")
    print("Camino a la solución:", resultado.solution())
else:
    print("No se ha encontrado una solución.")

Se ha encontrado una solución
Camino a la solución: [('movimiento', 3, 0, 3, 2), ('movimiento', 4, 1, 3, 1)]


##### Análisis de Búsquedas Ciegas
El análisis en esta nueva versión apenas cambia con respecto al anterior en cuanto a completitud y optimización, aunque, por ejemplo, en este caso la búsqueda en profundidad sin control de repetidos sí llega a la solución. La diferencia entre las dos versiones radica sobre todo en la eficiencia, siendo esta última la más eficiente de las dos.

### ANÁLISIS DE LAS VERSIONES CON PROBLEMA_CON_ANALIZADOS

In [32]:
class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados = 0  # Contador para el número de nodos analizados

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1  # Incrementa el contador de nodos analizados
        return self.problem.goal_test(estado)  # Usa el método goal_test del problema

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado, accion)


In [33]:

import time
def resuelve_MoVix_puzzle1(estado_inicial, algoritmo, h=None):
    puzzle = MovIX_Puzzle1(N=5, X=4, Y=1, initial=estado_inicial)
    puzzle_analizado = Problema_con_Analizados(puzzle)
    
    start_time = time.time()  # Iniciar cronómetro
    
    # Ejecuta el algoritmo directamente, sin verificar la solvencia
    if h: 
        sol = algoritmo(puzzle_analizado, h).solution()
    else: 
        sol = algoritmo(puzzle_analizado).solution()
    
    end_time = time.time()  # Terminar cronómetro
    elapsed_time = (end_time - start_time) * 1000  # Tiempo en milisegundos
    
    # Retorna los resultados en lugar de solo imprimirlos
    return {
        'Solución': sol,
        'Longitud': len(sol),
        'Nodos analizados': puzzle_analizado.analizados,
        'Tiempo (ms)': elapsed_time
    }


In [34]:

import time
def resuelve_MoVix_puzzle2(estado_inicial, algoritmo, h=None):
    puzzle = MovIX_Puzzle2(N=5, X=4, Y=1, initial=estado_inicial)
    puzzle_analizado = Problema_con_Analizados(puzzle)
    
    tiempo_ini = time.time()  # Iniciar cronómetro
    
    # Ejecuta el algoritmo directamente, sin verificar la solvencia
    if h: 
        sol = algoritmo(puzzle_analizado, h).solution()
    else: 
        sol = algoritmo(puzzle_analizado).solution()
    
    tiempo_fin = time.time()  # Terminar cronómetro
    tiempo = (tiempo_fin - tiempo_ini) * 1000  # Tiempo en milisegundos
    
    # Retorna los resultados en lugar de solo imprimirlos
    return {
        'Solución': sol,
        'Longitud': len(sol),
        'Nodos analizados': puzzle_analizado.analizados,
        'Tiempo (ms)': tiempo
    }


##### Tablas de datos

In [35]:
estado_inicial1 = (
 (' ', ' ', ' ', ' ', ' '),
 (' ', 'S', ' ', 'H', ' '),
 (' ', ' ', 'L', ' ', ' '),
 ('V', ' ', ' ', ' ', ' '),
 (' ', ' ', ' ',' ',' ')
)
estado_inicial2 = [
    ('S', 1, 1),  # Ficha 'S' en la fila 1, columna 1
    ('L', 2, 2),  # Ficha 'L' en la fila 2, columna 2
    ('V', 3, 0),  # Ficha 'V' en la fila 3, columna 0
    ('H', 1, 3)   # Ficha 'H' en la fila 1, columna 3
]


##### Búsqueda en profundidad con control de repetidos

In [36]:
import pandas as pd
resultado1  = resuelve_MoVix_puzzle1(estado_inicial1, depth_first_graph_search)
resultado2  = resuelve_MoVix_puzzle2(estado_inicial2, depth_first_graph_search)
# Crear un DataFrame
data = {
    'Implementación': ['Implementación 1', 'Implementación 2'],
    'Longitud': [resultado1['Longitud'], resultado2['Longitud']],
    'Nodos': [resultado1['Nodos analizados'], resultado2['Nodos analizados']],
    'tiempo': [resultado1['Tiempo (ms)'], resultado2['Tiempo (ms)']]
}

df = pd.DataFrame(data)

# Mostrar la tabla
df

,Implementación,Longitud,Nodos,tiempo
0,Implementación 1,98,1375,687.484980
1,Implementación 2,3,149,2.511501


##### Búsqueda en anchura sin control de repetidos

In [37]:
resultado1  = resuelve_MoVix_puzzle1(estado_inicial1, breadth_first_tree_search)
resultado2  = resuelve_MoVix_puzzle2(estado_inicial2, breadth_first_tree_search)
# Crear un DataFrame
data = {
    'Implementación': ['Implementación 1', 'Implementación 2'],
    'Longitud': [resultado1['Longitud'], resultado2['Longitud']],
    'Nodos': [resultado1['Nodos analizados'], resultado2['Nodos analizados']],
    'tiempo': [resultado1['Tiempo (ms)'], resultado2['Tiempo (ms)']]
}

df = pd.DataFrame(data)

# Mostrar la tabla
df

,Implementación,Longitud,Nodos,tiempo
0,Implementación 1,2,67,7.086992
1,Implementación 2,2,16,0.000000


##### Búsqueda en anchura con control de repetidos

In [38]:
resultado1  = resuelve_MoVix_puzzle1(estado_inicial1, breadth_first_graph_search)
resultado2  = resuelve_MoVix_puzzle2(estado_inicial2, breadth_first_graph_search)
# Crear un DataFrame
data = {
    'Implementación': ['Implementación 1', 'Implementación 2'],
    'Longitud': [resultado1['Longitud'], resultado2['Longitud']],
    'Nodos': [resultado1['Nodos analizados'], resultado2['Nodos analizados']],
    'tiempo': [resultado1['Tiempo (ms)'], resultado2['Tiempo (ms)']]
}

df = pd.DataFrame(data)

# Mostrar la tabla
df

,Implementación,Longitud,Nodos,tiempo
0,Implementación 1,2,57,4.350901
1,Implementación 2,2,16,0.000000


Hemos probado el problema con el estado inicial dado en el enunciado, en el caso más básico. A pesar de ser la instancia menos complicada del problema, vemos cómo los resultados varían drásticamente de una versión a otra:

- La **versión del tablero completo** requiere mucho más tiempo y más nodos, mientras que la **versión con estados más simples** no necesita tantos recursos.
- También observamos que la **búsqueda en anchura** resulta bastante más eficiente, siendo la mejor opción en cuanto a eficiencia la que tiene **control de repetidos**. 
  - En este caso, que es bastante simple, no notamos grandes diferencias entre tener o no control de repetidos, pero el algoritmo que lo tiene resulta más rápido y necesita algunos nodos menos en la primera versión.

### HEURÍSTICAS

Tenemos en cuenta que el coste de las fichas de moverse hacia cualquier casilla (no importa el número de casillas que se mueva) es 1. Esto es debido a que buscamos la **solución más óptima en cuanto al número de movimientos desde el inicio**, sin importar las casillas que tengamos que movernos con cada ficha.

#### Primera heurística: Cuenta las fichas que faltan para conseguir X fichas alineadas en cualquier dirección.

El algoritmo calcula, para cada ficha en el estado, el mínimo número de movimientos adicionales necesarios para completar una línea de cuatro fichas en cualquier dirección (horizontal, vertical o diagonal).  Devuelve el menor de estos valores, indicando cuántas fichas faltan para lograr una alineación ganadora


In [39]:
def h_1(node):
    estado = node.state
    N = 5
    X = 4
    posiciones_ocupadas = [(ficha, fila, columna) for (ficha, fila, columna) in estado]
    
    min_movs_para_alinear = X  # Asumimos inicialmente que se necesitan `X` fichas adicionales

    # Verificamos alineaciones parciales para cada ficha
    for ficha, fila, columna in posiciones_ocupadas:
        # Inicializamos contadores de alineación para cada dirección
        fichas_horizontal = 1
        fichas_vertical = 1
        fichas_diagonal_izq_der = 1
        fichas_diagonal_der_izq = 1

        # Comprobamos alineación horizontal
        for i in range(1, X):
            if columna + i < N and (ficha, fila, columna + i) in posiciones_ocupadas:
                fichas_horizontal += 1

        # Comprobamos alineación vertical
        for i in range(1, X):
            if fila + i < N and (ficha, fila + i, columna) in posiciones_ocupadas:
                fichas_vertical += 1

        # Comprobamos alineación diagonal de izquierda a derecha
        for i in range(1, X):
            if fila + i < N and columna + i < N and (ficha, fila + i, columna + i) in posiciones_ocupadas:
                fichas_diagonal_izq_der += 1

        # Comprobamos alineación diagonal de derecha a izquierda
        for i in range(1, X):
            if fila + i < N and columna - i >= 0 and (ficha, fila + i, columna - i) in posiciones_ocupadas:
                fichas_diagonal_der_izq += 1

        # Calcula movimientos adicionales necesarios en cada dirección
        min_movs_para_alinear = min(
            min_movs_para_alinear,
            X - fichas_horizontal,
            X - fichas_vertical,
            X - fichas_diagonal_izq_der,
            X - fichas_diagonal_der_izq
        )

    return min_movs_para_alinear


#### Segunda heurística: Calcula la distancia Manhattan desde la ficha a su casilla más cercana para formar una línea.

El algoritmo calcula una heurística sumando las distancias de Manhattan necesarias para que cada ficha alcance su posición objetivo según su tipo








In [40]:
def h_2(node):
   
    estado = node.state
    N = 5
    X = 4
    posiciones_ocupadas = [(ficha, fila, columna) for (ficha, fila, columna) in estado]
    
    manhattan_distancia = 0

    for ficha, fila, columna in posiciones_ocupadas:
        if ficha == 'V':
            objetivo_fila = (fila + (X - 1)) % N  # Posición alineada en la columna
            manhattan_distancia += abs(fila - objetivo_fila)
        elif ficha == 'H':
            objetivo_columna = (columna + (X - 1)) % N  # Posición alineada en la fila
            manhattan_distancia += abs(columna - objetivo_columna)
        elif ficha == 'S' or ficha == 'L':
            manhattan_distancia += sum(abs(f - fila) + abs(c - columna) for _, f, c in posiciones_ocupadas)

    return manhattan_distancia


#### Tercera heurística: Distancia Manhattan dividida entre N

Dividimos la distancia Manhattan entre N ya que la heurística anterior no resulta admisible para el coste 1 de las acciones.

In [41]:
def h_3(node):
    estado = node.state
    N = 5
    X = 4
    posiciones_ocupadas = [(ficha, fila, columna) for (ficha, fila, columna) in estado]
    
    manhattan_distancia = 0

    for ficha, fila, columna in posiciones_ocupadas:
        if ficha == 'V':
            objetivo_fila = (fila + (X - 1)) % N  # Posición alineada en la columna
            manhattan_distancia += abs(fila - objetivo_fila)
        elif ficha == 'H':
            objetivo_columna = (columna + (X - 1)) % N  # Posición alineada en la fila
            manhattan_distancia += abs(columna - objetivo_columna)
        elif ficha == 'S' or ficha == 'L':
            manhattan_distancia += sum(abs(f - fila) + abs(c - columna) for _, f, c in posiciones_ocupadas)
        manhattan_distancia= manhattan_distancia/N

    return manhattan_distancia

#### Cuarta heurística: Distancia Manhattan dividida según la ficha 

Dividimos la distancia Manhattan: las fichas V y H entre N (ya que pueden moverse a cualquier casilla de la vertical u horizontal), la ficha S entre N/2 (teniendo en cuenta su salto solo quedarán N/2 casillas a las que pueda llegar) y la ficha L no se divide puesto que puede ocupar cualquier casilla del tablero.

In [42]:
def h_4(node):
   
    estado = node.state
    N = 5
    X = 4
    posiciones_ocupadas = [(ficha, fila, columna) for (ficha, fila, columna) in estado]
    
    manhattan_distancia = 0

    for ficha, fila, columna in posiciones_ocupadas:
        if ficha == 'V':
            objetivo_fila = (fila + (X - 1)) % N  # Posición alineada en la columna
            manhattan_distancia += abs(fila - objetivo_fila)
            manhattan_distancia = manhattan_distancia/N
        elif ficha == 'H':
            objetivo_columna = (columna + (X - 1)) % N  # Posición alineada en la fila
            manhattan_distancia += abs(columna - objetivo_columna)
            manhattan_distancia = manhattan_distancia/N
        elif ficha == 'S' or ficha == 'L':
            manhattan_distancia += sum(abs(f - fila) + abs(c - columna) for _, f, c in posiciones_ocupadas)
            if ficha == 'S':
                manhattan_distancia = manhattan_distancia/(N/2)

    return manhattan_distancia


#### Quinta heurística: Combinación de las tres heurísticas anteriores.

Combinación de la primera heurística (las fichas que faltan para la alineación) con la heurística más informada de la distancia Manhattan.

In [43]:
def h_5(node):
    return h_1(node) + h_4(node) 

#### Análisis con heurísticas sobre la segunda versión

In [44]:
# Definir el estado inicial
e1 = [
    ('S', 1, 1),  # Ficha 'S' en la fila 1, columna 1
    ('L', 2, 3),  # Ficha 'L' en la fila 2, columna 3
    ('V', 3, 0),  # Ficha 'V' en la fila 3, columna 0
    ('H', 1, 3)   # Ficha 'H' en la fila 1, columna 3
]


# Crear una instancia del problema
puzzle = MovIX_Puzzle2(N=5, X=4, Y=1, initial=e1)


Usamos la busqueda A*

In [45]:
%%time
astar_search(puzzle, h_5).solution()


CPU times: total: 0 ns
Wall time: 1.97 ms


[('movimiento', 2, 3, 1, 2), ('movimiento', 3, 0, 1, 0)]

#### Tabla con datos de tiempo usando las heurísticas

In [80]:
import time
import pandas as pd

resultados_heuristicas = []
heuristicas = [
    ("Heurística 1", h_1),
    ("Heurística 2", h_2),
    ("Heurística 3", h_3),
    ("Heurística 4", h_4),
    ("Heurística 5", h_5),
]
for nombre, heuristica in heuristicas:
    inicio = time.time()  
    resultado = astar_search(puzzle, h = heuristica).solution() 
    fin = time.time()  
    
    tiempo_ejecucion = (fin - inicio)*1000
    
    resultados_heuristicas.append((nombre, tiempo_ejecucion))


df_tiempos = pd.DataFrame(resultados_heuristicas, columns=["Heurística", "Tiempo (milisegundos)"])

df_tiempos


,Heurística,Tiempo (milisegundos)
0,Heurística 1,0.973225
1,Heurística 2,0.000000
2,Heurística 3,2.491236
3,Heurística 4,0.972271
4,Heurística 5,1.951694


#### Propiedades de la heurística:
- **Heurística 1** (número de fichas que faltan para la alineación)
    - **Admisibilidad**: Es admisible puesto que si el número de fichas que faltan para la alineación son x, habrá que hacer mínimo x movimientos para colocar las fichas (1 por cada ficha por lo menos).
    - **Consistencia**: Cuando de un nodo a uno de sus descendientes se reduce la heurística en 1 significa que se ha alineado una ficha más y esto solo ocurrirá si se da como poco 1 movimiento (con coste 1). Así para alinear 2 fichas necesitaremos mínimo 2 movimientos, para alinear 3 necesitaremos mínimo 3 movimientos, etc. Comprobamos entonces que es consistente y se cumple que h(ni) – h(nj) <= coste(ni,nj).
    - **Eficiencia**: Es poco eficiente con respecto a las búsquedas ciegas, ya que debemos recorrer el tablero en cada nodo e ir buscando alineaciones.

- **Heurística 2** (distancia Manhattan)
    - **Admisibilidad**: No es admisible dado un coste 1 para cualquier movimiento independientemente del número de casillas. Esto sucede porque la distancia Manhattan para la alieación de una ficha vertical puede ser de 3 casillas (el valor de la heurística sería 3) pero tendremos que el coste real es 1.
    - **Consistencia**: Tampoco es consistente. De la misma manera, dado un nodo i el coste hacia cada uno de sus descendientes inmediatos será 1 mientras que la heurística puede haberse disminuido en nj en más de una unidad si hemos conseguido alinear una ficha que estuviera a distancia Manhattan mayor que 1.
    - **Eficiencia**: Sin embargo, vemos que es una heurística bastante eficiente en cuanto al tiempo requerido en la búsqueda de la solución.

- **Heurística 3** (distancia Manhattan/N)
    --> Es una variante de la heurística anterior para hacerla **admisible**.
    - **Admisibilidad**: Sin tocar el coste de las acciones, conseguimos que la heurística sea admisible ya que dado cualquier nodo el valor heurístico de este anteriormente era a lo sumo N (como mucho la ficha está separada N casillas de donde debe acabar--> distancia Manhattan), dividiendo el valor entre N tendríamos entonces que el valor heurístico de un movimiento nunca será mayor que 1, por lo que será siempre menor que el coste real.
    - **Consistencia**: Con la medida tomada, el coste real de ir de un nodoo i a un sucesor nj será como mínimo 1, mientras que la variación del valor heurístico de ni a nj será como máximo 1 en caso de que se haya movido una ficha que estuviera a N casillas de su objetivo. De esta manera, en este caso se cumple el requisito de consistencia y, dado que se cumple en el caso de los extremos, se cumple de igual manera para el resto de casos.
    - **Eficiencia**: A pesar de que ahora la heurística es admisible, resulta realmente ineficiente en cuanto al tiempo.
    
- **Heurística 4** (distancia Manhattan/según ficha)
    --> Es una variante de la heurística anterior para hacerla **más informada**.
    - **Admisibilidad**: Al igual que la anterior es admisible, ahora teniendo en cuenta que el valor heurístico de la distancia Manhattan original era a lo sumo N en los movimientos de fichas verticales y horizontales; en el caso de las saltadoras la distancia original sería como máximo N/2 puesto que siempre debían saltar 1 casilla por movimiento; en el caso de las fichas lentas no dividiremos la distancia Manhattan ya que el coste de llegar a la casilla objetivo será igual al número de casillas que nos marca la esta.
    - **Consistencia**: Se sigue cumpliendo la condición de consistencia.
    - **Eficiencia**: Es más efectiva en cuanto al tiempo que la variante de la distancia Manhattan hecha en la heurística anterior pero no más que la original.
- **Heurística 5** (combinación de 1 y 4)
    - **Admisibilidad**: Dado que las heurísticas 1 y 4 son admisibles, tendremos que está también lo es.
    - **Consistencia**: Es consistente como las heurísticas que combinamos.
    - **Eficiencia**: Es mucho más eficiente que cada una por separado en cuanto al tiempo.

#### Tabla de nodos y longitud de la solución usando heurísticas

In [47]:
def resuelve_MoVix_puzzle2(estado_inicial, algoritmo, h=None):
    puzzle = MovIX_Puzzle2(N=5, X=4, Y=1, initial=estado_inicial)
    puzzle_analizado = Problema_con_Analizados(puzzle)
    
    start_time = time.time()  # Iniciar cronómetro
    
    # Ejecuta el algoritmo y verifica si devuelve una solución válida
    if h: 
        resultado = algoritmo(puzzle_analizado, h)
    else: 
        resultado = algoritmo(puzzle_analizado)
    
    end_time = time.time()  # Terminar cronómetro
    elapsed_time = (end_time - start_time) * 1000  # Tiempo en milisegundos
    
    if resultado:  # Verificar si hay un resultado válido
        sol = resultado.solution()
        return {
            'Solución': sol,
            'Longitud': len(sol),
            'Nodos analizados': puzzle_analizado.analizados,
            'Tiempo (ms)': elapsed_time
        }

Análisis de eficiencia de nuestras heurísticas

In [82]:
N = 5
Y = 1
# Definir el estado inicial
e1 = [
    ('S', 1, 1),  # Ficha 'S' en la fila 1, columna 1
    ('L', 2, 3),  # Ficha 'L' en la fila 2, columna 3
    ('V', 3, 0),  # Ficha 'V' en la fila 3, columna 0
    ('H', 1, 3)   # Ficha 'H' en la fila 1, columna 3
]


heuristicas = [h_1, h_2, h_3, h_4, h_5]  


In [83]:
# Inicializar la lista vacía en cada ejecución
resultados = []

for i, heuristica in enumerate(heuristicas, 1):
    resultado = resuelve_MoVix_puzzle2(e1, astar_search, heuristica)
    if resultado:  
        resultado['Heurística'] = f'Heurística {i}'  
        resultados.append(resultado)

# Crear el DataFrame
df = pd.DataFrame(resultados, columns=['Heurística', 'Longitud', 'Nodos analizados', 'Tiempo (ms)'])

# Mostrar la tabla
df


,Heurística,Longitud,Nodos analizados,Tiempo (ms)
0,Heurística 1,2,6,0.972748
1,Heurística 2,2,3,0.000000
2,Heurística 3,2,16,4.522085
3,Heurística 4,2,9,0.846148
4,Heurística 5,2,9,1.952410
